# Governor Effect on COVID-19 Progression

## Final Project for Knowledge Discovery in Databases course

In [2]:
# Import libraries
import pandas as pd
import numpy as np

In [131]:
# Read in Data

# Johns Hopkins Data: Daily Cases
# https://github.com/CSSEGISandData/COVID-19
US_07_24 = pd.read_csv("Johns_Hopkins_Data/Daily_Reports/07-24-2020_US.csv")

# Governors data
govn = pd.read_csv("Other_Data/GOVERNORS.csv")

# US Census State Populations
pop = pd.read_csv("Other_Data/CENSUS_State_Populations.csv")
pop = pop[["NAME", "POPESTIMATE2019"]]

# US Census State Area Measurements
areas = pd.read_csv("Other_Data/CENSUS_StateAreaMeasurements.csv")
areas = areas[['State_and_Other_Areas', 'TotalArea_SqMi', 'LandArea_SqMi']]

# Median Household Income per State
hhi = pd.read_csv("Other_Data/HH_Income.csv")

# State GDP
gdp = pd.read_csv("Other_Data/State_GDP.csv")

# Stay-At-Home Order Dates
stay = pd.read_csv("Other_Data/State_Stay_At_Home_Dates.csv")

# Population Distribution by Age
agedistrib = pd.read_csv("Other_Data/Population_Distribution_by_Age.csv")

In [132]:
# Merge data sets
df01 = pd.merge(US_07_24, govn, how='inner', left_on = 'Province_State', right_on = 'State')
df02 = pd.merge(df01, pop, how='inner', left_on='Province_State', right_on='NAME')
df03 = pd.merge(df02, areas, how='inner', left_on='Province_State', right_on='State_and_Other_Areas')
df04 = pd.merge(df03, hhi, how='inner', left_on='Province_State', right_on='Location')
df05 = pd.merge(df04, gdp, how='inner', left_on='Province_State', right_on='State')
df06 = pd.merge(df05, stay, how='inner', left_on='Province_State', right_on='State')
df07 = pd.merge(df06, agedistrib, how='inner', left_on='Province_State', right_on='Location')

df = df07

In [122]:
# Calculate Deaths per Million
df["DeathsPerMillion"]=df.apply(lambda row: row["Deaths"]/(row["POPESTIMATE2019"]/1000000), axis=1)

# Calculate Confirmed Cases per Million
# NOTE: This is actually not needed because Incidence_Rate from the original data set is cases/100K people
# df06["ConfirmedPerMillion"]=df06.apply(lambda row: row["Confirmed"]/(row["POPESTIMATE2019"]/1000000), axis=1)

# Calculate Population Density
df["Population_Density"]=df.apply(lambda row: row["POPESTIMATE2019"]/row["LandArea_SqMi"], axis=1)

In [123]:
# Convert data columns to dates
df["Last_Update_Converted"]=pd.to_datetime(df["Last_Update"])
df["DOB_Converted"]=pd.to_datetime(df["DOB"])
df["Inauguration_Date_Converted"]=pd.to_datetime(df["Inauguration_Date"])
df["SAHO_Effective_Date_Converted"]=pd.to_datetime(df["SAHO_Effective_Date"])

# Governor's Age (in YEARS) at time of first COVID-19 case in the U.S. (January 21, 2020)
df["Gov_Age_at_Onset"] = df.apply(lambda row: (pd.to_datetime("2020-01-21") - row["DOB_Converted"])/np.timedelta64(1,'Y'), axis=1)

# How long had Governor been in office (in MONTHS) at time of first COVID-19 case in the U.S. (January 21, 2020)
df["Gov_Tenure_at_Onset"] = df.apply(lambda row: (pd.to_datetime("2020-01-21") - row["Inauguration_Date_Converted"])/np.timedelta64(1,'M'), axis=1)

In [124]:
# IMPUTATION

# If a Stay-At-Home Order was never issued for a state, impute with the date of the Johns Hopkins snapshot. 
# This will effectively communicate that no order was issued
df["SAHO_Effective_Date_Converted"].replace(np.NaN, pd.to_datetime("2020-07-24"), inplace=True)

In [125]:
# Column containing number of days between first U.S. case and Stay-At-Home Order
df["SAHO_Response_Time"] = df.apply(lambda row: (row["SAHO_Effective_Date_Converted"] - pd.to_datetime("2020-01-21"))/np.timedelta64(1,'D'), axis=1)


In [126]:
# Print information on dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Province_State                 50 non-null     object        
 1   Country_Region                 50 non-null     object        
 2   Last_Update                    50 non-null     object        
 3   Lat                            50 non-null     float64       
 4   Long_                          50 non-null     float64       
 5   Confirmed                      50 non-null     int64         
 6   Deaths                         50 non-null     int64         
 7   Recovered                      44 non-null     float64       
 8   Active                         50 non-null     float64       
 9   FIPS                           50 non-null     int64         
 10  Incident_Rate                  50 non-null     float64       
 11  People_Tested        

In [130]:
# Export merged table to csv to investigate
df.to_csv("Other_Data/FULL_TABLE.csv")

In [127]:
# Reduce width of dataframe by selecting desired columns
columns = ['Province_State', 'Confirmed', 'Deaths', 'Incident_Rate', 'DeathsPerMillion', 'People_Tested', 
           'Testing_Rate', 'Sex', 'Party', 'POPESTIMATE2019', 'TotalArea_SqMi', 'Med_HHI_thousands', 'GDP_Billions',
          'Pct_65Plus', 'Population_Density', 'Gov_Age_at_Onset', 'Gov_Tenure_at_Onset', 'SAHO_Response_Time']

subset = df[columns]

In [128]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Province_State       50 non-null     object 
 1   Confirmed            50 non-null     int64  
 2   Deaths               50 non-null     int64  
 3   Incident_Rate        50 non-null     float64
 4   DeathsPerMillion     50 non-null     float64
 5   People_Tested        50 non-null     float64
 6   Testing_Rate         50 non-null     float64
 7   Sex                  50 non-null     object 
 8   Party                50 non-null     object 
 9   POPESTIMATE2019      50 non-null     int64  
 10  TotalArea_SqMi       50 non-null     int64  
 11  Med_HHI_thousands    50 non-null     float64
 12  GDP_Billions         50 non-null     float64
 13  Pct_65Plus           50 non-null     int64  
 14  Population_Density   50 non-null     float64
 15  Gov_Age_at_Onset     50 non-null     float

#### Modeling

In [ ]:
# First Model Concept

# Linear Regression 
# Dependent Variable:  Incident_Rate
# Predictor Variables: Testing_Rate, Sex, Party, Population_Density, Med_HHI_thousands, GDP_Billions, Pct_65Plus,
#                      Gov_Age_at_Onset, Gov_Tenure_at_Onset, SAHO_Response_Time



# Second Model Concept 

# Same as first but with DeathsPerMillion as Dependent Variable